## 1. 사용환경 준비
### 1.1 필요 라이브러리 설치
`pip install python-dotenv langchain langchain-openai faiss-cpu pypdf`

.env 파일에 OPENAI_API_KEY 를 지정해 준 뒤 사용함

In [3]:
from dotenv import (
    load_dotenv,
)  # dotenv 모듈 : .env 파일을 읽고 여기서 정의된 환경 변수를 시스템의 환경변수로 설정함
import os

load_dotenv()  # .env 파일에서 환경 변수 로드

api_key = os.getenv(
    "OPENAI_API_KEY"
)  # 환경변수에서 OPENAI_API_KEY 의 값 가져와 api_key 변수에 저장
print(api_key)  # API KEY 가 제대로 입력됐는지 확인

k-proj-l8wPUQs3PxMzl9psPzXWSlFXgiNAfQGtTOFaQj2FGzQNZS4KlKT6c18eHs2lp8gS8-Mp-dBbgkT3BlbkFJuA9b79_enpgtDcllStNzkCoBqRoCGlkNx7LLzH4-nsu5yw0-D8vhI6qTJQTNz25dYDXrNzekYA


## 2. 모델 로드하기

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import (
    HumanMessage,
)  # Messages 작성(사용자의 요구사항이나 질문)을 위한 모듈 import

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

## 3. 문서 로드하기
`pip install pypdf`

PyPDFLoader : PDF 파일을 로드하고, 이를 문서로 변환. 이 문서들은 후속 작업에서 사용할 수 있도록 텍스트로 변환됨

In [5]:
from langchain.document_loaders import PyPDFLoader  #

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader(
    "/Users/t2023-m0072/Desktop/assignment_LLM_RAG/RAG/PDF/초거대 언어모델 연구 동향 (1).pdf"
)

# 페이지 별 문서 로드
docs = loader.load()

# 로드된 문서 출력
for doc in docs:
    print(doc.page_content)

8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이
에 따라 사용자의 목적에 맞는 출력을 생성하는 패러
다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난 
발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며, 이 요인들은 현대 자연언어
처리 (Natural Language Processing, NLP) 연구의 핵심
적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는 
무시할 수 없는 중요한 요인이다. 디지털화의 선도로, 
텍스트 데이터의 양이 기하급수적으로 증가하였고, 
이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의 
활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥
락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째
로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정
적이었다. 특히, Graphics Processing Unit (GPU) 및 
Tensor Processing Unit (TPU) 와 같은 고성능 병렬 처
리 하드웨어의 개발은 모델 학습에 있어 병목 현상을 
크게 완화시켰다. 이로 인해 연구자들은 모델의 복잡
성을 키우고, 더욱 깊은 신경망 구조를 탐구할 수 있
게 되었다. 셋째, 알고리즘 및 기술의 발전은 LLM의 
성능 향상을 주도하였다. Attention 및 Transformer 
Architecture의 도입은 연구자들에게 

## 4. 문서 청크로 나누기

### 4.0 Chunk VS Token
#### Chunk
: 문자나 단어 등의 더 큰 의미 단위로 텍스트를 나누는 방식. 문맥적 의미를 기준으로 나눈다
* 특징
  * 크기 : 문자수 (characters) 나 단어수 (words) 로 정의됨
  * 의미 단위 : 청크는 텍스트를 의미가 있는 단위로 나누는데 중점을 둠. 
    * 문장 단위
    * 문단 단위
    * 구문 단위 등

#### Token
: 텍스트의 기본적인 처리단위. NLP 모델에서 텍스트를 벡터화 하거나 분석하기 전에 텍스트를 나누는 가장 작은 단위. 모델이 이해할 수 있는 기본적인 단위
* 토큰화 (Tokenization) : 텍스트를 단어, 구두점, 특수 문자 등으로 분할하는 과정
* 특징
  * 크기 : 고정되어 있지 않음. 단어, 하위 단어, 구두점 등 다양한 요소로 구성됨
  * 언어 모델의 기본 단위 : GPT 같은 언어 모델은 토큰을 입력 받아 처리함

## 4.1 CharacterTextSplitter
Langchain 에서 제공하는 문서 분할 도구. 문자 단위로 텍스트를 나눔.
### 4.1.1 CharacterTextSplitter 특징 
* **고정된 청크 크기** : 텍스트를 동일한 크기로 나눔. 일관성 있는 처리에 유용
* **속도** : 단순한 방식 덕분에 처리 속도가 빠르고, 대규모 데이터셋을 처리할 때 효율적
* **단순성** : 구현이 간단. 복잡한 설정 없이 다양한 응용 프로그램에 쉽게 통합
* 중복 오버랩 (Overlap) : 각 청크 간 겹치는 부분을 두어 텍스트를 자연스럽게 나눌 수 있음. 중복을 추가해 나눠진 청크가 더 의미 있는 단위로 나뉘어 모델이 문맥을 더 잘 이해할 수 있음


### 4.1.2 parameter
* `separator` : 텍스트를 나누는 기준을 설정
  * 기본값 : 공백 `' '`
* `chunk_size` : 각 분할된 청크의 최대 길이를 설정
  * `chunk_size=100` : 각 청크가 최대 100자 까지 포함됨을 의미
* `chunk_overlap` : 청크 간 곂치는 문자 수를 설정  -> 각 텍스트간 의미가 끊어지지 않도록 하기 위함
  * `chunk_overlap=10` : 각 청크가 이전 청크와 최대 10자 겹침
* `length_function` : 각 청크의 길이 계산 함수
  * `length_function=len` : 각 청크의 길이가 문자 수로 계산됨. 
  * `length_function=lambda text: len(text.split())` : 단어 수로 텍스트 나눔
* `is_separator_regex` : separator 가 정규표현식 인지
  * `is_separator_regex=False`(기본값) : separator로 지정된 텍스트가 정확히 일치하는 부분에서 텍스트를 나눔
  *  `is_separator_regex=True` : separator 에 정규 표현식을 사용할 수 있음
     *  `separator=r'\n+'`를 사용하면 여러 줄 바꿈이 있더라도 이를 하나의 분할 기준으로 사용

In [24]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",  # 분할기준 : 두 줄 바꿈을 기준으로 텍스트 나누기
    chunk_size=100,  # 청크 크기 : 최대 100자
    chunk_overlap=10,  # 중복 오버랩 : 각 청크가 최대 10자 겹침
    length_function=len,  # 길이 계산 함수 : 문자 수 기준 (len())
    is_separator_regex=False,  # 구분자가 정규 표현식인지 여부 (False : 정확히 일치하는 구분자 기준)
)

splits = text_splitter.split_documents(docs)

# 청킹된 내용 상위 10개 출력
top_10 = splits[:10]
# print(f"Chunk 3:{splits[3]}")
for i, chunk in enumerate(top_10, 1):
    page_content = chunk.page_content
    print(f"Chunk {i}:{page_content[:300]}")
# print(f"splits 길이 : {len(splits)}")

Chunk 1:8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모
Chunk 2:2023. 11 정보과학회지 9
사 결정을 위해 LLM을 활용하는 경우에 문제가 될 
수 있다. 또한, LLM 을 악의적인 목적으로 사용하는 
위험성도 있다 [10]. 예를 들면, 미스리딩 정보 생성이 
나 편향된 정보 전파를 위한 도구로 활용될 수 있다. 
이 외에도 LLM의 동작 원리나 결과에 대한 설명력 
부족, 최신 정보를 반영하는 데의 한계 등 여러 문제
점이 있으며, 이러한 문제점들을 해결하는 것은 다가
오는 연구의 중요한 도전 과제로 여겨진다.
즉 편향성 (LLM은 학습 데이터에 포함된 편향을 
반영할 수 있음), 안전
Chunk 3:10 특집원고  초거대 언어모델 연구 동향
치하는 정보들을 기억하지 못하는 장기 의존성 문제
가 존재한다. 이러한 문제를 극복하기 위해, LSTM 
(Long Short-Term Memory) [14] 과 GRU (Gated 
Recurrent Unit) [15] 가 등장했다. 하지만, 이들은 모두 
텍스트에 존재하는 단방향 문맥 정보만 활용한다는 
한계를 지닌다.
양방향 문맥 정보를 활용하기 위해, ELMo [16] 는 
주어진 텍스트에 존재하는 순방향 문맥 정보와 역방
향 문맥 정보를 함께 활용하는 양방향 학습을 제안했
다. 이
Chunk 4:2023. 11 정보과학회지 11
국어를 비효율적으로 토큰화하고, 학습한 한국어 토
큰 수가 매우 부족하다는 한계를 

## 4.2 RecursiveCharacterTextSplitter
내용의 **문맥을 유지**하면서 텍스트를 관리 가능한 청크로 분할 하도록 하는 청킹 방식. 대량의 텍스트를 처리할 때 유용. 관련 정보들이 서로 인접하게 유지되므로 **가독성**과 **이해도**를 높이는데 효과적.
### 4.2.1 RecursiveCharacterTextSplitter 특징
* **계층적 세분화** : 텍스트를 문맥에 나누어 의미가 각 청크 간에 보존됨
* **적응형 청크 크기** : 텍스트 내용에 따라 청크 크기가 달라질 수 있음. 길이가 다른 문단이 포함된 문서에 유용
* **문맥 보존** : 주변 텍스트를 고려해 청킹 과정 중 중요한 정보가 손실되는 것을 최소화 함

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 각 청크 최대 100자 까지
    chunk_overlap=10, # 청크간 최대 10자 중복
    length_function=len, # 길이 계산 함수 : 문자 수 기준 (len())
    is_separator_regex=False, # 구분자가 정규 표현식인지 여부 (False : 정확히 일치하는 구분자 기준)
)

splits = recursive_text_splitter.split_documents(docs)

# 청킹된 내용의 상위 10개 출력
top_10 = splits[:10]
# print(f"Chunk 3:{splits[3]}")
for i, chunk in enumerate(top_10, 1):
    page_content = chunk.page_content
    print(f"Chunk {i}:{page_content[:300]}")

Chunk 1:8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
Chunk 2:1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
Chunk 3:던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
Chunk 4:응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이
Chunk 5:에 따라 사용자의 목적에 맞는 출력을 생성하는 패러
다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난
Chunk 6:발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며, 이 요인들은 현대 자연언어
Chunk 7:처리 (Natural Language Processing, NLP) 연구의 핵심
적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는
Chunk 8:무시할 수 없는 중요한 요인이다. 디지털화의 선도로, 
텍스트 데이터의 양이 기하급수적으로 증가하였고, 
이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의
Chunk 9:활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥
락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째
로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정
Chunk 10:적이었다. 특히, Graphics Processing Unit (GPU) 및 
Tensor Processing Unit (TPU) 와 같은 고성능 병렬 처


## 5. 벡터 임베딩 생성

Chunk 1:8 특집원고  초거대 언어모델 연구 동향
초거대 언어모델 연구 동향
업스테이지  박찬준*･이원성･김윤기･김지후･이활석
 
1. 서  론1)
Chunk 2:1. 서  론1)
ChatGPT1)와 같은 초거대 언어모델(Large Language 
Model, LLM) 의 등장으로 기존에 병렬적으로 연구되
Chunk 3:던 다양한 자연언어처리 하위 분야들이 하나의 모델
로 처리되고 있으며, 태스크 수렴 현상 (Converge)이 
발생하고 있다. 즉 하나의 LLM으로 번역, 요약, 질의
Chunk 4:응답, 형태소분석 등의 작업을 모두 처리할 수 있게 
되었다. 프롬프트 (Prompt)를 어떻게 모델에게 입력하
느냐에 따라서 LLM의 다양한 능력들이 창발되고, 이
Chunk 5:에 따라 사용자의 목적에 맞는 출력을 생성하는 패러
다임을 맞이하게 되었다 [1].
LLM은 최근 몇 년 간의 연구 동향에 따라 뛰어난
Chunk 6:발전을 이루고 있다. 이러한 발전은 몇 가지 주요한 
요인에 기반하고 있으며, 이 요인들은 현대 자연언어
Chunk 7:처리 (Natural Language Processing, NLP) 연구의 핵심
적인 추세로 간주된다. 첫째로, 데이터의 양적 확대는
Chunk 8:무시할 수 없는 중요한 요인이다. 디지털화의 선도로, 
텍스트 데이터의 양이 기하급수적으로 증가하였고, 
이는 연구의 질적 변화를 가져왔다. 대규모 코퍼스의
Chunk 9:활용은 LLM의 일반화 능력을 향상시키며, 다양한 맥
락과 주제에 대한 깊은 학습을 가능하게 한다. 둘째
로, 컴퓨팅 기술의 진보는 LLM의 발전에 있어 결정
Chunk 10:적이었다. 특히, Graphics Processing Unit (GPU) 및 
Tensor Processing Unit (TPU) 와 같은 고성능 병렬 처


## 6. 벡터 스토어 생성

## 7. FAISS 를 Retriever 로 변환

## 8. 프롬프트 템플릿 정의

## 9. RAG 체인 구성

## 10. 챗봇 구동 확인